In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
from random import randint

from IPython.display import Image
from IPython.core.display import Image, display

%load_ext autoreload
%autoreload 2

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
# tb=SummaryWriter()
torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [4]:
bs = 32
# Load Data
train_dataset = datasets.ImageFolder(root='./No_Image_Preprocessing_Pattern1/Training/Normal', transform=transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(), 
]))
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=False)
len(train_dataset.imgs), len(train_dataloader)
train_dataset

Dataset ImageFolder
    Number of datapoints: 256
    Root location: ./No_Image_Preprocessing_Pattern1/Training/Normal
    StandardTransform
Transform: Compose(
               Resize(size=128, interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )

In [5]:
# Fixed input for debugging
fixed_x, _ = next(iter(train_dataloader))
# save_image(fixed_x, 'VAE_CLAHE_Training.png')

# Image('VAE_CLAHE_Training.png')

In [6]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [7]:
class UnFlatten(nn.Module):
    def forward(self, input, size=1024):
        return input.view(input.size(0), size, 1, 1)

In [8]:
class VAE(nn.Module):
    def __init__(self, image_channels=3, h_dim=1024, z_dim=64):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=2, stride=2),
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=2, stride=2),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=2, stride=2),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=2, stride=2),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=2, stride=2),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(512, 1024, kernel_size=2, stride=2),
            nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.Conv2d(1024, 1024, kernel_size=2, stride=2),
            nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            Flatten()
        )
        
        self.fc1 = nn.Linear(h_dim, z_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)
        
        self.decoder = nn.Sequential(
            UnFlatten(),
            nn.ConvTranspose2d(1024, 1024, kernel_size=2, stride=2),
            nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=2, stride=2),
            nn.BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.Sigmoid(),
        )
        
    def reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        # return torch.normal(mu, std)
        esp = torch.randn(*mu.size())
        z = mu + std * esp
        return z
    
    def bottleneck(self, h):
        mu, logvar = self.fc1(h), self.fc2(h)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def encode(self, x):
        h = self.encoder(x)
        z, mu, logvar = self.bottleneck(h)
        return z, mu, logvar

    def decode(self, z):
        z = self.fc3(z)
        z = self.decoder(z)
        return z

    def forward(self, x):
        z, mu, logvar = self.encode(x)
        z = self.decode(z)
        return z, mu, logvar

In [9]:
image_channels = fixed_x.size(1)
image_channels

3

In [10]:
model = VAE(image_channels=image_channels)
model.to(device)

VAE(
  (encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(2, 2), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 64, kernel_size=(2, 2), stride=(2, 2))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(64, 128, kernel_size=(2, 2), stride=(2, 2))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(128, 256, kernel_size=(2, 2), stride=(2, 2))
    (10): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Conv2d(256, 512, kernel_size=(2, 2), stride=(2, 2))
    (13): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Conv2d(512, 1024, kernel_size=(2, 2), stride=(2, 2))
    (16): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_run

In [11]:
def loss_fn(recon_x, x, mu, logvar):
#     BCE = F.binary_cross_entropy(recon_x, x, size_average=False).to(device)
    MSE = F.mse_loss(recon_x, x, reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp()).to(device)
    return MSE + KLD, MSE, KLD

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [13]:
def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 3, 128, 128)
    return x

# Testing

In [14]:
from IPython.display import Image
from IPython.core.display import Image, display

In [15]:
bs = 32
# Load Data
test_dataset = datasets.ImageFolder(root='./No_Image_Preprocessing_Pattern1/Test/N/', transform=transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(), 
]))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=bs, shuffle=False)
len(test_dataset.imgs), len(test_dataloader)
test_dataset

Dataset ImageFolder
    Number of datapoints: 128
    Root location: ./No_Image_Preprocessing_Pattern1/Test/N/
    StandardTransform
Transform: Compose(
               Resize(size=128, interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )

In [16]:
# Fixed input for debugging
fixed_test, _ = next(iter(test_dataloader))
# save_image(fixed_test, 'xxx.png')

# Image('xxx.png')

In [17]:
dataloader_iterator = iter(test_dataloader)
for i in range(len(test_dataloader)):
    try:
        data, target = next(dataloader_iterator)
        save_image(data, './No_Image_Preprocessing_Pattern1/Test_Reconstructed/Testing_Original_VAE_latent64_{}.png'.format(i))
    except StopIteration:
        dataloader_iterator = iter(dataloader)
        data, target = next(dataloader_iterator)

In [18]:
test= VAE(image_channels=image_channels)
test.to(device)

VAE(
  (encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(2, 2), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 64, kernel_size=(2, 2), stride=(2, 2))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(64, 128, kernel_size=(2, 2), stride=(2, 2))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(128, 256, kernel_size=(2, 2), stride=(2, 2))
    (10): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Conv2d(256, 512, kernel_size=(2, 2), stride=(2, 2))
    (13): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Conv2d(512, 1024, kernel_size=(2, 2), stride=(2, 2))
    (16): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_run

In [19]:
test.load_state_dict(torch.load('./autoencoder_VAE_latent64_Thesis_Pattern1.pth'))
test.eval()

VAE(
  (encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(2, 2), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 64, kernel_size=(2, 2), stride=(2, 2))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(64, 128, kernel_size=(2, 2), stride=(2, 2))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(128, 256, kernel_size=(2, 2), stride=(2, 2))
    (10): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Conv2d(256, 512, kernel_size=(2, 2), stride=(2, 2))
    (13): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Conv2d(512, 1024, kernel_size=(2, 2), stride=(2, 2))
    (16): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_run

In [20]:
b=-1
for idx, (images, _) in enumerate(test_dataloader):
    images, _=images.cuda(), _.cuda()
    recon_images, mu, logvar = test(images)
    loss, bce, kld = loss_fn(recon_images, images, mu, logvar)
    pic = to_img(recon_images.cpu().data)
    b=b+1
    save_image(pic, './No_Image_Preprocessing_Pattern1/Test_Reconstructed/Testing_Reconstructed_Normal_VAE_latent64_{}.png'.format(b))
#     print('loss:{:.4f}'.format(loss.data))

In [21]:
bs = 1
# Load Data
test_dataset = datasets.ImageFolder(root='./No_Image_Preprocessing_Pattern1/Test/N/', transform=transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(), 
]))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=bs, shuffle=False)
len(test_dataset.imgs), len(test_dataloader)
test_dataset

Dataset ImageFolder
    Number of datapoints: 128
    Root location: ./No_Image_Preprocessing_Pattern1/Test/N/
    StandardTransform
Transform: Compose(
               Resize(size=128, interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )

In [22]:
# Fixed input for debugging
fixed_test, _ = next(iter(test_dataloader))
# save_image(fixed_test, 'xxx.png')

# Image('xxx.png')

In [23]:
loss_append=[]
for idx, (images, _) in enumerate(test_dataloader):
    images, _=images.cuda(), _.cuda()
    recon_images, mu, logvar = test(images)
    loss, bce, kld = loss_fn(recon_images, images, mu, logvar)
#     print('{:.4f}'.format(loss.data))
    loss = round(float(loss.data),4)
    loss_append.append(loss)

print(loss_append)

[15.8491, 77.1413, 75.3286, 17.2044, 15.0334, 20.3347, 17.5572, 17.4297, 20.083, 19.3995, 18.1543, 81.4189, 79.2332, 18.8133, 71.541, 18.1932, 20.2143, 21.2451, 19.3407, 13.228, 18.4447, 14.8377, 13.6319, 15.1786, 17.082, 10.1755, 12.2737, 14.8983, 15.8664, 16.9394, 19.7474, 73.941, 16.1861, 26.1337, 20.6428, 20.0689, 22.2314, 18.5795, 87.4217, 91.4237, 72.0904, 71.2395, 81.5008, 94.5164, 102.1982, 125.9195, 15.9733, 16.7832, 15.3908, 18.0892, 16.186, 14.1012, 15.9085, 16.5495, 20.8201, 22.4088, 25.4558, 34.617, 59.3532, 39.2752, 91.7926, 30.756, 34.3481, 61.1353, 51.1695, 44.2684, 123.8496, 53.6164, 20.5175, 22.1885, 36.207, 64.9779, 130.9507, 101.68, 151.9765, 122.8367, 126.9604, 168.1967, 143.2027, 141.6596, 83.0098, 72.9478, 69.7442, 78.4675, 90.8466, 95.6865, 124.4812, 12.5025, 12.0596, 8.6646, 9.2384, 12.3093, 11.9592, 12.3277, 17.0029, 88.2335, 10.3854, 9.4816, 18.0457, 98.2792, 12.0842, 10.3696, 10.8427, 9.9182, 9.5914, 12.646, 78.5984, 80.4492, 15.7467, 19.1806, 82.2455, 22.93

In [25]:
bs = 32
# Load Data
test_dataset = datasets.ImageFolder(root='./No_Image_Preprocessing_Pattern1/Test/A/', transform=transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(), 
]))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=bs, shuffle=False)
len(test_dataset.imgs), len(test_dataloader)
test_dataset

Dataset ImageFolder
    Number of datapoints: 128
    Root location: ./No_Image_Preprocessing_Pattern1/Test/A/
    StandardTransform
Transform: Compose(
               Resize(size=128, interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )

In [26]:
# Fixed input for debugging
fixed_test, _ = next(iter(test_dataloader))
# save_image(fixed_test, 'xxx.png')

# Image('xxx.png')

In [27]:
dataloader_iterator = iter(test_dataloader)
for i in range(len(test_dataloader)):
    try:
        data, target = next(dataloader_iterator)
        save_image(data, './No_Image_Preprocessing_Pattern1/Test_Reconstructed/Testing_Abnormal_VAE_latent64_{}.png'.format(i))
    except StopIteration:
        dataloader_iterator = iter(dataloader)
        data, target = next(dataloader_iterator)

In [28]:
b=-1
for idx, (images, _) in enumerate(test_dataloader):
    images, _=images.cuda(), _.cuda()
    recon_images, mu, logvar = test(images)
    loss, bce, kld = loss_fn(recon_images, images, mu, logvar)
    pic = to_img(recon_images.cpu().data)
    b=b+1
    save_image(pic, './No_Image_Preprocessing_Pattern1/Test_Reconstructed/Testing_Reconstructed_Abnormal_VAE_latent64_{}.png'.format(b))
#     print('loss:{:.4f}'.format(loss.data))

In [29]:
bs = 1
# Load Data
test_dataset = datasets.ImageFolder(root='./No_Image_Preprocessing_Pattern1/Test/A/', transform=transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(), 
]))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=bs, shuffle=False)
len(test_dataset.imgs), len(test_dataloader)
test_dataset

Dataset ImageFolder
    Number of datapoints: 128
    Root location: ./No_Image_Preprocessing_Pattern1/Test/A/
    StandardTransform
Transform: Compose(
               Resize(size=128, interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )

In [30]:
# Fixed input for debugging
fixed_test, _ = next(iter(test_dataloader))
# save_image(fixed_test, 'xxx.png')

# Image('xxx.png')

In [31]:
loss_append_ano=[]
for idx, (images, _) in enumerate(test_dataloader):
    images, _=images.cuda(), _.cuda()
    recon_images, mu, logvar = test(images)
    loss, bce, kld = loss_fn(recon_images, images, mu, logvar)
#     print('{:.4f}'.format(loss.data))
    loss = round(float(loss.data),4)
    loss_append_ano.append(loss)
print(loss_append_ano)

[209.3277, 169.9041, 227.5607, 141.8554, 303.6899, 271.8334, 213.589, 90.7801, 211.1326, 174.3153, 168.4976, 152.004, 180.2031, 156.0409, 118.792, 170.6975, 1031.7002, 412.3208, 452.2428, 641.0493, 1395.7559, 425.3205, 716.5728, 380.0307, 85.6064, 501.28, 5.77607560111063e+16, 1803671650172928.0, 4.144746519855104e+17, 6.817256944887631e+21, 35500224151552.0, 9461670346752.0, 5059283779584.0, 241773104332800.0, 1581084180480.0, 72888288.0, 216300240.0, 36185411584.0, 2622529847951360.0, 5.398131363964518e+16, 2325566193664.0, 129591040.0, 519.9814, 646.563, 499.1664, 129.6111, 168.4323, 88.8074, 60.5398, 108.248, 137.9653, 123.2534, 67.687, 147.5929, 124.6011, 169.188, 191.1688, 378.5792, 271.6268, 188.7611, 136.0882, 204.3412, 167.2565, 42.7295, 44283.9727, 187924.4844, 2021296570368.0, 1443279.25, 4214270464.0, 12249.0176, 37791.7852, 3718.9971, 399.0495, 376.5633, 719.7583, 1189.6281, 1131.5093, 2805.0356, 2297.2886, 412.6993, 9.589618120912783e+27, 1182.9854, 82256896.0, 1259.8999,

In [32]:
import numpy as np
percentile=np.percentile(loss_append, 95, axis=0)
percentile

125.41609499999998

# Anomaly

In [33]:
normal_testing=loss_append
normal_testing = np.array_split(normal_testing, len(normal_testing)/32)

anomaly_testing=loss_append_ano
anomaly_testing = np.array_split(anomaly_testing, len(anomaly_testing)/32)

a=np.array(normal_testing).shape
a[0]

4

In [34]:
#TP
TP=0
FN=0
for j in range(128):   
    if loss_append_ano[j]>percentile:   
        TP=TP+1
    else:
        FN=FN+1

TN=0
FP=0
for j in range(128):   
    if loss_append[j]>percentile:   
        FP=FP+1
    else:
        TN=TN+1
print ("TN=",TN,"TP=",TP)
print ("FP=",FP,"FN=",FN)

TN= 121 TP= 117
FP= 7 FN= 11


In [35]:
percentile=np.percentile(loss_append, 90, axis=0)
percentile
#TP
TP=0
FN=0
for j in range(128):   
    if loss_append_ano[j]>percentile:   
        TP=TP+1
    else:
        FN=FN+1

TN=0
FP=0
for j in range(128):   
    if loss_append[j]>percentile:   
        FP=FP+1
    else:
        TN=TN+1
print ("TN=",TN,"TP=",TP)
print ("FP=",FP,"FN=",FN)

TN= 115 TP= 121
FP= 13 FN= 7


In [36]:
normal_testing=loss_append
normal_testing = np.array_split(normal_testing, len(normal_testing)/32)

anomaly_testing=loss_append_ano
anomaly_testing = np.array_split(anomaly_testing, len(anomaly_testing)/32)

a=np.array(normal_testing).shape
a[0]

4

In [37]:
print("Normal Testing:")
for b in range(a[0]):
    print('\nNormal Batch{}'.format(b+1))
    for i in range(32):
        if (i+1)%8!=0:
            print(round(normal_testing[b][i],5),end=" ")
        else:
            print(round(normal_testing[b][i],5))

print("\nAnomaly Testing:")
for c in range(a[0]):
    print('\nAnomaly Batch{}'.format(c+1))
    for z in range(32):
        if (z+1)%8!=0:
            print(round(anomaly_testing[c][z],5),end=" ")
        else:
            print(round(anomaly_testing[c][z],5))
b=0
c=0
i=0
z=0

Normal Testing:

Normal Batch1
15.8491 77.1413 75.3286 17.2044 15.0334 20.3347 17.5572 17.4297
20.083 19.3995 18.1543 81.4189 79.2332 18.8133 71.541 18.1932
20.2143 21.2451 19.3407 13.228 18.4447 14.8377 13.6319 15.1786
17.082 10.1755 12.2737 14.8983 15.8664 16.9394 19.7474 73.941

Normal Batch2
16.1861 26.1337 20.6428 20.0689 22.2314 18.5795 87.4217 91.4237
72.0904 71.2395 81.5008 94.5164 102.1982 125.9195 15.9733 16.7832
15.3908 18.0892 16.186 14.1012 15.9085 16.5495 20.8201 22.4088
25.4558 34.617 59.3532 39.2752 91.7926 30.756 34.3481 61.1353

Normal Batch3
51.1695 44.2684 123.8496 53.6164 20.5175 22.1885 36.207 64.9779
130.9507 101.68 151.9765 122.8367 126.9604 168.1967 143.2027 141.6596
83.0098 72.9478 69.7442 78.4675 90.8466 95.6865 124.4812 12.5025
12.0596 8.6646 9.2384 12.3093 11.9592 12.3277 17.0029 88.2335

Normal Batch4
10.3854 9.4816 18.0457 98.2792 12.0842 10.3696 10.8427 9.9182
9.5914 12.646 78.5984 80.4492 15.7467 19.1806 82.2455 22.938
76.7249 97.8096 103.5784 95.8553 8